In [1]:
%pip install llama_stack_client fire

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 201.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 219.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [15]:
from llama_stack_client import RAGDocument, LlamaStackClient

MODEL_NAME = "granite-40-h-1b"
BASE_URL = "http://lsd-llama-inference-only-service-walkguardianai-llm.apps.cluster-pzdb5.pzdb5.sandbox5281.opentlc.com" 


client = LlamaStackClient(base_url=BASE_URL)
models = client.models.list()



INFO:httpx:HTTP Request: GET http://lsd-llama-inference-only-service-walkguardianai-llm.apps.cluster-pzdb5.pzdb5.sandbox5281.opentlc.com/v1/models "HTTP/1.1 200 OK"


In [19]:
for model in models:
    print(model)

Model(identifier='granite-40-h-1b', metadata={}, api_model_type='llm', provider_id='vllm-inference', type='model', provider_resource_id='granite-40-h-1b', model_type='llm')
Model(identifier='granite-embedding-125m', metadata={'embedding_dimension': 768.0}, api_model_type='embedding', provider_id='sentence-transformers', type='model', provider_resource_id='ibm-granite/granite-embedding-125m-english', model_type='embedding')


In [20]:
model_id = next(m.identifier for m in models if m.model_type == "llm")
model_id

'granite-40-h-1b'

In [33]:
transcript = """
[00:14] “Someone's following us…”
[00:17] “No, seriously, he’s getting closer.”
[00:21] I'm going to kill you! You better run.
"""

response = client.inference.chat_completion(
    model_id="granite-40-h-1b",
    messages=[
        {
            "role": "system",
            "content": """
            You are a safety-analysis model. Your task is to determine whether the user is currently in danger based on an audio transcript.

            The input will be a transcript containing:
            - Time-stamped segments  
            - Spoken words from multiple speakers  
            - Nonverbal cues when available (e.g., footsteps, glass breaking, screams)
            
            Your analysis must be highly accurate, objective, and concise.
            
            ### OUTPUT FORMAT (strict):
            danger_level: <1–10>
            danger_type: <one of the categories below>
            summary: <1–3 sentences explaining the reasoning>
            recommended_action: <clear, practical, and safe advice>
            
            ### Danger Level Scale:
            1–3: low risk (normal conversation, no signs of danger)
            4–6: moderate risk (concerning cues; monitor the situation)
            7–8: high risk (possible immediate threat)
            9–10: extreme risk (urgent danger; may require emergency action)
            
            ### Allowed Danger Types:
            - medical_distress
            - physical_threat
            - stalking_or_following
            - domestic_dispute
            - verbal_aggression
            - possible_theft
            - intoxication_or_impairment
            - lost_or_disoriented
            - mental_health_crisis
            - environmental_hazard (e.g., fire, car traffic, loud crashes)
            - unknown (if not enough information)
            
            Your output must always be structured exactly as specified, with no additional commentary.
            
            """
        },
        {
            "role": "user",
            "content": transcript
        }
    ]
)

print(response.completion_message.content)

INFO:httpx:HTTP Request: POST http://lsd-llama-inference-only-service-walkguardianai-llm.apps.cluster-pzdb5.pzdb5.sandbox5281.opentlc.com/v1/inference/chat-completion "HTTP/1.1 200 OK"


danger_level: 8
danger_type: physical_threat
summary: The user expresses a clear intention to harm someone, using strong language and a threat to kill. The rapid escalation of the conversation and the explicit threat indicate a high level of danger.
recommended_action: Immediately take evasive action, seek help from authorities, and avoid the area. Do not confront the person or engage in further dialogue.
